In [78]:
import pandas as pd
import sqlite3

In [79]:
def run_query(q):
    """Creates a connector object to a predifined sqlite3 database using a context manager and returns a pandas Dataframe

    Args:
        q (string): A string containing the SQL query

    Returns:
        pandas.DataFrame: Returns a query as a Pandas.DataFrame object
    """
    with sqlite3.connect('chinook.db') as conn:
        return pd.read_sql(q, conn)

def run_command(c):
    """Creates a connector object to a predifined sqlite3 database \n
    using a context manager and executes SQL commands that do not produce output.

    Args:
        c (string): A string containing the SQL query or sqlite command
    """
    with sqlite3.connect('chinook.db') as conn:
        conn.isolation_level = None
        conn.execute(c)

def show_tables():
    """ Uses the function run_query to display the tables and schema of the database.\n
     Analog to the command (.tables) in the sqlite3 console
    """

    results = run_query(
                """
                SELECT
                name,
                type
                FROM sqlite_master
                WHERE type IN ("table","view");
                """
            )
    return results

# Test functions and database connection

In [80]:
run_query("SELECT * FROM invoice LIMIT 5;")

,invoice_id,customer_id,invoice_date,billing_address,billing_city,billing_state,billing_country,billing_postal_code,total
0,1,18,2017-01-03 00:00:00,627 Broadway,New York,NY,USA,10012-2612,15.84
1,2,30,2017-01-03 00:00:00,230 Elgin Street,Ottawa,ON,Canada,K2P 1L7,9.90
2,3,40,2017-01-05 00:00:00,"8, Rue Hanovre",Paris,None,France,75002,1.98
3,4,18,2017-01-06 00:00:00,627 Broadway,New York,NY,USA,10012-2612,7.92
4,5,27,2017-01-07 00:00:00,1033 N Park Ave,Tucson,AZ,USA,85719,16.83


In [81]:
show_tables()

,name,type
0,album,table
1,artist,table
2,customer,table
3,employee,table
4,genre,table
5,invoice,table
6,invoice_line,table
7,media_type,table
8,playlist,table
9,playlist_track,table


# Summary of purchases made by Genre in the USA only.

Some new released albums are going to be introduced into the companies stock. But not all of them could be sold well in the US market. Therefore, based on past sales records, it would be interesting to find out which genres are best-selling in the US (of course, without taking into account the Artist's popularity, which might stand out from less popular genres).

In [82]:
query = """
WITH raw_summary AS (
    SELECT *
    FROM 
        track AS t
    INNER JOIN genre AS g ON g.genre_id = t.genre_id
    INNER JOIN invoice_line AS il ON il.track_id = t.track_id
    INNER JOIN invoice AS i ON i.invoice_id = il.invoice_id
    INNER JOIN customer AS c ON c.customer_id = i.customer_id
),
    summary AS
(
    SELECT 
    rs.country,
    g.name,
    COUNT(rs.track_id) number_of_tracks,
        (
            SELECT SUM(rsa.quantity)
            FROM raw_summary AS rsa
            GROUP BY rsa.country
            HAVING rsa.country LIKE 'USA'
        ) AS total

    FROM raw_summary AS rs
    LEFT JOIN genre AS g ON g.genre_id = rs.genre_id
    GROUP BY 1,2
    HAVING rs.country LIKE 'USA'
)

SELECT
    name as genre,
    number_of_tracks AS tracks_sold,
    ROUND(CAST(number_of_tracks AS float) / CAST(total AS float),4) * 100 AS tracks_sold_percent
    FROM summary
    ORDER BY 3 DESC;
"""

tracks_sold_usa = run_query(query)
# tracks_sold_usa


In [83]:
import plotly.express as px
import plotly.io as pio

pio.templates.default = 'ggplot2'



In [84]:

fig = px.bar(
    tracks_sold_usa, 
    x='genre', 
    y='tracks_sold_percent',  
    height=400, width=600,
    title = 'Sales Volume  by Genre',
    labels = dict(genre='Music genre', tracks_sold_percent="Tracks sold")
    
    )
fig.update_layout(margin=dict(l=10, r=20, t=40, b=10))
fig.show(renderer="notebook_connected")

The results show that the Rock Genre contributes to over 50\% of the sales volume in the USA. Making it by far, the most successful and lucrative.
Derivates of Rock like Metal, Alternative \& Punk account for about 23\% of the sales volume followed by R&B/Soul and Blues with only about 8\%

Taking into account the new releases which where picked for consideration by the producing label (see table), and having to pick only 3

| Artist Name          | Genre   |
|----------------------|---------|
| Regal                | Hip-Hop |
| Red Tone             | Punk    |
| Meteor and the Girls | Pop     |
| Slim Jim Bites       | Blues   |

It would be wise to choose Punk, Pop and Blues. Although Pop and Hip-Hop have very similar sales values and this could easily change in the future. Giving a more accurate result could not be possible without looking at surveys or statistics of radio stations about the popularity of pop or Hip Hop. 

# Comparing Sales Agents performance

In [85]:
query = """

SELECT 
    e.first_name || ' ' || e.last_name AS name,
    e.hire_date,
    SUM(ii.total) total_sales
FROM employee AS e
INNER JOIN customer AS c ON c.support_rep_id = e.employee_id
INNER JOIN invoice AS ii ON ii.customer_id = c.customer_id
GROUP BY 1, 2 ORDER BY 3 DESC;
"""
employee_performance = run_query(query)
employee_performance

,name,hire_date,total_sales
0,Jane Peacock,2017-04-01 00:00:00,1731.51
1,Margaret Park,2017-05-03 00:00:00,1584.00
2,Steve Johnson,2017-10-17 00:00:00,1393.92


In [86]:
employee_performance_temp = employee_performance.copy()
employee_performance_temp['name'] = employee_performance_temp['name'].apply(lambda x: x.replace(' ', '<br>'))
# this is just slightly modifyed version of the results so that the name column can be reformatted including an html brake for plotting.

import datetime as dt

fig = px.line(employee_performance_temp, x="hire_date", y="total_sales", 
                text='name', 
                height=400, 
                width=800, 
                title='Sales Performance in correlation with hiring date',
                labels = dict(hire_date="Hire date", total_sales="Total Sales")
                )
fig.update_traces(textposition='top right')
fig.update_layout(margin=dict(l=10, r=20, t=50, b=10))
fig.update_xaxes(range= [dt.date(2017,3, 20), dt.date(2017,11, 15)])
fig.update_yaxes(range= [min(employee_performance_temp.total_sales) -100 , max(employee_performance_temp.total_sales)+100])
fig.show(renderer="notebook_connected")


Looking at the line graph between the total sales and the hire date, we can see there is a negative correlation. This can be explained since employees that where hired more recently also generated less revenue since we are tanking into account the records from all year and evaluating them at the same time. This can be misleading since we actually would like to know the performance in a comparable metric. One way to do this would be to instead evaluate the average sales amount per customer. This metric is then dependent, not on the amount of customers that the employee had in a year. But the average revenue generated per customer order.

In [87]:
query = """

SELECT 
    e.first_name || ' ' || e.last_name AS name,
    e.hire_date,
    ROUND(AVG(ii.total), 2) AS average_sales,
    COUNT(ii.invoice_id) AS no_sales
FROM employee AS e
INNER JOIN customer AS c ON c.support_rep_id = e.employee_id
INNER JOIN invoice AS ii ON ii.customer_id = c.customer_id
GROUP BY 1, 2 ORDER BY 2 DESC;
"""
employee_performance_avg = run_query(query)
employee_performance_avg

,name,hire_date,average_sales,no_sales
0,Steve Johnson,2017-10-17 00:00:00,7.41,188
1,Margaret Park,2017-05-03 00:00:00,7.40,214
2,Jane Peacock,2017-04-01 00:00:00,8.17,212


This new metric shows that on average, all employees generate similar amounts of revenue. One could also take a look at the number of time a customer placed an order per employee ``no_sales`` and relate this to the average sales revenue ``average``. We can observe that ``Steve Johnson`` which is the most recent hire, has in average a slightly higher revenue per customer order than ``Margaret Park``. But he achieved that performance with less customer orders, wich implies that Steve is able to sell more expensive products or bigger orders. 

In [88]:
query = '''

WITH
one_customers AS
(
    select COUNT(customer_id) nr_customers, country from customer
    group by 2
    HAVING nr_customers = 1
),
new_customer AS
(
    SELECT *,
       CASE
        WHEN (country IN (SELECT country from one_customers)) THEN 'Other'
        ELSE country
        END AS new_country
        FROM customer
),
new_customer_sorted AS
(
    SELECT *, 
        CASE
            WHEN new_country = "Other" THEN 1
            ELSE 0
        END AS sort from new_customer
)

SELECT ncs.new_country country,
       COUNT(DISTINCT ncs.customer_id) Total_customers,
       ROUND(SUM(i.total), 2) Total_sales,
       ROUND(SUM(i.total) / COUNT(DISTINCT ncs.customer_id), 2) avg_sales_per_customer,
       ROUND(SUM(i.total) /  COUNT(i.invoice_id), 2) avg_order_value
    
FROM new_customer_sorted AS ncs
INNER JOIN invoice i ON i.customer_id=ncs.customer_id
GROUP BY 1
ORDER BY ncs.sort ASC
'''

Sales_by_country = run_query(query)
Sales_by_country

,country,Total_customers,Total_sales,avg_sales_per_customer,avg_order_value
0,Brazil,5,427.68,85.54,7.01
1,Canada,8,535.59,66.95,7.05
2,Czech Republic,2,273.24,136.62,9.11
3,France,5,389.07,77.81,7.78
4,Germany,4,334.62,83.66,8.16
5,India,2,183.15,91.58,8.72
6,Portugal,2,185.13,92.57,6.38
7,USA,13,1040.49,80.04,7.94
8,United Kingdom,3,245.52,81.84,8.77
9,Other,15,1094.94,73.00,7.45


In [91]:
fig = px.scatter(Sales_by_country,
                 x="Total_customers", 
                 y="Total_sales", 
                 size="avg_sales_per_customer", 
                 color="country", 
                 title="Sales Volume by Country",
                 labels = dict(Total_customers="Number of customers", Total_sales="Total Sales")
                 )

fig.update_layout(margin=dict(l=10, r=20, t=50, b=10))
fig.show(renderer="iframe")